In [ ]:
pip install --upgrade polars

In [ ]:
#importing all libraries
import pandas as pd
import numpy as np
from google.colab import drive
import shutil
import glob
import gc
import csv
import os
import datetime
import json
from urllib.request import urlopen
import pprint
import polars as pl
#from zipfile import ZipFile

In [ ]:
shutil.rmtree('/content/sample_data')
drive.mount('/content/drive')

Mounted at /content/drive


# Intro

**Algorithm for Tableau** (to match limitations of public version)

**Main idea:** aggregate up to number of violations

**Additional assumption:** Time span to be limited to 12 months. Assumption: preprocessing is made on the whole dataset and the final result is limited to chosen date range.

Columns of the input dataset:
```
['plate_id', 'plate_type', 'registration_state', 'vehicle_body_type', 'violation_description', 'violation_precinct', 'violation_county', 'house_number', 'street_name', 'issue_datetime']
```
Steps:
1. 'plate_id': drop
2. 'plate_type': aggretate to top3 or 5 and other map as other
3. 'registration_state': aggretate to top3 or 5 and other map as other
4. 'vehicle_body_type': keep aggregation as is
5. 'violation_description': let's try to map to its sense (long|short, not parking) or just aggregate to top-10
6. 'violation_precinct': check for validity accross counties and other drop
7. 'violation_county': keep
8. 'house_number'; drop as we cannot locate it with long|lat
9. 'street_name': keep top-15 or in every precinct
10. 'issue_datetime': split into date and time rounded to hour

Then aggregate up to the number of violations

In [ ]:
# download as Polars DataFrame

features = {
    # 'plate_id':'str',
    'registration_state':pl.Categorical,
    'plate_type':pl.Categorical,
    # 'Violation Code':'categorical',
    'vehicle_body_type':pl.Categorical,
    'violation_description': pl.Categorical,
    #'Street Code1':'int32',
    #'Street Code2':'int32',
    #'Street Code3':'int32',
    'violation_precinct': pl.UInt16,
    'violation_county':pl.Categorical,
    'house_number':str,
    'street_name':str,
    'issue_datetime': pl.Datetime
}

path = '/content/drive/MyDrive/DataAnalyst/Hackathon/'
output_merged_file = 'data_source.csv'

dfpl = pl.read_csv(path+output_merged_file, columns=list(features.keys()), dtypes=features)
print('Est. memory_usage, mb: ', int(dfpl.estimated_size()/ 1024 / 1024) )
print(dfpl)

Est. memory_usage, mb:  1890
shape: (33_932_135, 9)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ plate_typ ┆ registrat ┆ vehicle_b ┆ violation ┆ … ┆ violation ┆ house_num ┆ street_na ┆ issue_da │
│ e         ┆ ion_state ┆ ody_type  ┆ _descript ┆   ┆ _county   ┆ ber       ┆ me        ┆ tetime   │
│ ---       ┆ ---       ┆ ---       ┆ ion       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
│ cat       ┆ cat       ┆ cat       ┆ ---       ┆   ┆ cat       ┆ str       ┆ str       ┆ datetime │
│           ┆           ┆           ┆ cat       ┆   ┆           ┆           ┆           ┆ [μs]     │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ PAS       ┆ NY        ┆ Suburban  ┆ 46-Double ┆ … ┆ Manhattan ┆ 712       ┆ 175th st  ┆ 2013-08- │
│           ┆           ┆           ┆ Parking   ┆   ┆           ┆           ┆           ┆ 04       │
│           ┆           ┆           ┆ (

# Mapping

## Plate type

In [ ]:
dfpl['plate_type'].value_counts(sort=True).select(
    pl.col("plate_type"),
    pl.col("count"),
    pl.col('count').map_elements(lambda x: x/dfpl.shape[0]).round(4).alias("percent_count"),
    pl.col('count').map_elements(lambda x: x/dfpl.shape[0]).cum_sum().round(4).alias("percent_cumm"),
).head(15)


plate_type,count,percent_count,percent_cumm
cat,u32,f64,f64
"""PAS""",24017971,0.7078,0.7078
"""COM""",7463627,0.22,0.9278
"""OMT""",879686,0.0259,0.9537
"""SRF""",285610,0.0084,0.9621
"""OMS""",275961,0.0081,0.9703
"""IRP""",221618,0.0065,0.9768
"""999""",167137,0.0049,0.9817
"""TRC""",103487,0.003,0.9848
"""MOT""",82558,0.0024,0.9872


Top 15 `plate_types` cover 99.5% of all occurances. We will map them and the rest to `other`

In [ ]:
plate_type_dict = {
    'PAS': 'Passenger',
    'COM': 'Commercial',
    'OMT': 'Omnibus',
    'SRF': 'Special',
    'OMS': 'Omnibus',
    'IRP': 'International',
    'TRC': 'Tractor',
    'MOT': 'Motorcycle',
    'OMR': 'Omnibus',
    'ORG': 'Commercial',
    'MED': 'Medical',
    'OML': 'Omnibus'
}

dfpl = dfpl.with_columns(pl.col('plate_type').cast(pl.Utf8).replace(plate_type_dict, default='other', return_dtype=pl.Categorical))

<ipython-input-4-0500e8f7df94>:16: CategoricalRemappingWarning: Local categoricals have different encodings, expensive re-encoding is done to perform this merge operation. Consider using a StringCache or an Enum type if the categories are known in advance
  dfpl = dfpl.with_columns(pl.col('plate_type').cast(pl.Utf8).replace(plate_type_dict, default='other', return_dtype=pl.Categorical))


In [ ]:
print('Est. memory_usage, mb: ', int(dfpl.estimated_size()/ 1024 / 1024) )
dfpl.head(5)

Est. memory_usage, mb:  1894


plate_type,registration_state,vehicle_body_type,violation_description,violation_precinct,violation_county,house_number,street_name,issue_datetime
cat,cat,cat,cat,u16,cat,str,str,datetime[μs]
"""Passenger""","""NY""","""Suburban""","""46-Double Park…",33,"""Manhattan""","""712""","""175th st""",2013-08-04 07:52:00
"""Commercial""","""NY""","""Van""","""46-Double Park…",33,"""Manhattan""","""201""","""177th st""",2013-08-04 12:40:00
"""Commercial""","""NY""","""Pickup""","""46-Double Park…",33,"""Manhattan""","""520""","""163rd st""",2013-08-05 12:43:00
"""Commercial""","""NY""","""Van""","""46-Double Park…",33,"""Manhattan""","""517""","""176th st""",2013-08-05 14:32:00
"""Passenger""","""NJ""","""Pickup""","""14-No Standing…",33,"""Manhattan""","""525""","""175th st""",2013-08-11 18:17:00


## Registration state

In [ ]:
dfpl['registration_state'].value_counts(sort=True).select(
    pl.col("registration_state"),
    pl.col("count"),
    pl.col('count').map_elements(lambda x: x/dfpl.shape[0]).round(4).alias("percent_count"),
    pl.col('count').map_elements(lambda x: x/dfpl.shape[0]).cum_sum().round(4).alias("percent_cumm"),
).head(15)

registration_state,count,percent_count,percent_cumm
cat,u32,f64,f64
"""NY""",26103695,0.7693,0.7693
"""NJ""",3291929,0.097,0.8663
"""PA""",852419,0.0251,0.8914
"""CT""",493915,0.0146,0.906
"""FL""",428982,0.0126,0.9186
"""MA""",309348,0.0091,0.9277
"""IN""",277854,0.0082,0.9359
"""VA""",231801,0.0068,0.9428
"""MD""",193144,0.0057,0.9485


In TOP 5 states, in addition to NY, we have NJ, PA, CT and MA that share border with New York. Let's map them and leave other as `other`

In [ ]:
registration_dict = {
    'NY': 'New York',
    'NJ': 'New Jersey',
    'PA': 'Pennsylvania',
    'CT': 'Connecticut',
    'MA': 'Massachusetts '
}

dfpl = dfpl.with_columns(pl.col('registration_state').cast(pl.Utf8).replace(registration_dict, default='other', return_dtype=pl.Categorical))

<ipython-input-8-69535ebca7ac>:9: CategoricalRemappingWarning: Local categoricals have different encodings, expensive re-encoding is done to perform this merge operation. Consider using a StringCache or an Enum type if the categories are known in advance
  dfpl = dfpl.with_columns(pl.col('registration_state').cast(pl.Utf8).replace(registration_dict, default='other', return_dtype=pl.Categorical))


In [ ]:
print('Est. memory_usage, mb: ', int(dfpl.estimated_size()/ 1024 / 1024) )
dfpl.head(5)

Est. memory_usage, mb:  1898


plate_type,registration_state,vehicle_body_type,violation_description,violation_precinct,violation_county,house_number,street_name,issue_datetime
cat,cat,cat,cat,u16,cat,str,str,datetime[μs]
"""Passenger""","""New York""","""Suburban""","""46-Double Park…",33,"""Manhattan""","""712""","""175th st""",2013-08-04 07:52:00
"""Commercial""","""New York""","""Van""","""46-Double Park…",33,"""Manhattan""","""201""","""177th st""",2013-08-04 12:40:00
"""Commercial""","""New York""","""Pickup""","""46-Double Park…",33,"""Manhattan""","""520""","""163rd st""",2013-08-05 12:43:00
"""Commercial""","""New York""","""Van""","""46-Double Park…",33,"""Manhattan""","""517""","""176th st""",2013-08-05 14:32:00
"""Passenger""","""New Jersey""","""Pickup""","""14-No Standing…",33,"""Manhattan""","""525""","""175th st""",2013-08-11 18:17:00


## Violation description

In [ ]:
with pl.Config(set_fmt_str_lengths=50, set_tbl_rows=30):
    # cfg.set_fmt_str_lengths=100
    # cfg.set_tbl_rows=30
    display(
        dfpl['violation_description'].value_counts(sort=True).select(
            pl.col("violation_description").cast(pl.Utf8),
            pl.col("count"),
            pl.col('count').map_elements(lambda x: x/dfpl.shape[0]).round(4).alias("percent_count"),
            pl.col('count').map_elements(lambda x: x/dfpl.shape[0]).cum_sum().round(4).alias("percent_cumm"),
    ).head(40)
    )

violation_description,count,percent_count,percent_cumm
str,u32,f64,f64
"""21-No Parking (street clean)""",5684051,0.1675,0.1675
"""38-Failure to Display Muni Rec""",4748736,0.1399,0.3075
"""14-No Standing""",3473546,0.1024,0.4098
"""37-Expired Muni Meter""",2755433,0.0812,0.491
"""20-No Parking (Com Plate)""",2341966,0.069,0.5601
"""46-Double Parking (Com Plate)""",2050460,0.0604,0.6205
"""20A-No Parking (Non-COM)""",2039138,0.0601,0.6806
"""40-Fire Hydrant""",1876396,0.0553,0.7359
"""19-No Stand (bus stop)""",1151262,0.0339,0.7698


In [ ]:
violations_dict = {
    '21-No Parking (street clean)': 'No parking',
    '38-Failure to Display Muni Rec': 'Unpaid',
    '14-No Standing': 'No standing',
    '37-Expired Muni Meter': 'Expired',
    '20-No Parking (Com Plate)': 'No parking',
    '46-Double Parking (Com Plate)': 'Wrong place',
    '20A-No Parking (Non-COM)': 'No parking',
    '40-Fire Hydrant': 'Wrong place',
    '19-No Stand (bus stop)': 'Wrong place',
    '70-Reg. Sticker Missing (NYS)': 'Not applicable',
    '69-Failure to Disp Muni Recpt': 'Unpaid',
    '16-No Std (Com Veh) Com Plate': 'No standing',
    '31-No Stand (Com. Mtr. Zone)': 'No standing',
    '47-Double PKG-Midtown': 'Wrong place',
    '50-Crosswalk': 'Wrong place',
    '17-No Stand (exc auth veh)': 'No standing',
    '42-Exp. Muni-Mtr (Com. Mtr. Z)': 'Expired',
    '48-Bike Lane': 'Wrong place',
    '84-Platform lifts in low posit': 'Not applicable' ,
    '51-Sidewalk': 'Wrong place',
    '78-Nighttime PKG on Res Street': 'No parking',
    '24-No Parking (exc auth veh)': 'No parking',
    '98-Obstructing Driveway': 'Wrong place',
    '10-No Stopping': 'No standing',
    '82-Unaltered Commerc Vehicle': 'Not applicable',
    '53-Safety Zone': 'Wrong place',
    '13-No Stand (taxi stand)': 'No standing',
    '67-Blocking Ped. Ramp': 'Wrong place',
    '85-Storage-3 hour Commercial': 'Long-term parking',
    '45-Traffic Lane': 'Wrong place',
    '68-Not Pkg. Comp. w Psted Sign': 'Not applicable',
    '66-Detached Trailer': 'Long-term parking',
    '18-No Stand (bus lane)': 'Wrong place',
    '77-Parked Bus (exc desig area)': 'Wrong place',
    '72-Insp Stkr Mutilated': 'Not applicable',
    '83-Improper Registration': 'Not applicable',
    '64-No STD Ex Con/DPL, D/S Dec': 'No standing',
    '61-Wrong Way': 'Not applicable',
    '60-Angle Parking': 'Not applicable',
    '23-No Parking (taxi stand)': 'No parking'
}

dfpl = dfpl.with_columns(pl.col('violation_description').cast(pl.Utf8).replace(violations_dict, default='other', return_dtype=pl.Categorical))


<ipython-input-11-ec01a7a597d8>:44: CategoricalRemappingWarning: Local categoricals have different encodings, expensive re-encoding is done to perform this merge operation. Consider using a StringCache or an Enum type if the categories are known in advance
  dfpl = dfpl.with_columns(pl.col('violation_description').cast(pl.Utf8).replace(violations_dict, default='other', return_dtype=pl.Categorical))


## Delete unnecessary rows

In [ ]:
# Not applicable violations

dfpl = dfpl.filter(~(pl.col('violation_description')=='Not applicable'))
dfpl

plate_type,registration_state,vehicle_body_type,violation_description,violation_precinct,violation_county,house_number,street_name,issue_datetime
cat,cat,cat,cat,u16,cat,str,str,datetime[μs]
"""Passenger""","""New York""","""Suburban""","""Wrong place""",33,"""Manhattan""","""712""","""175th st""",2013-08-04 07:52:00
"""Commercial""","""New York""","""Van""","""Wrong place""",33,"""Manhattan""","""201""","""177th st""",2013-08-04 12:40:00
"""Commercial""","""New York""","""Pickup""","""Wrong place""",33,"""Manhattan""","""520""","""163rd st""",2013-08-05 12:43:00
"""Commercial""","""New York""","""Van""","""Wrong place""",33,"""Manhattan""","""517""","""176th st""",2013-08-05 14:32:00
"""Passenger""","""New Jersey""","""Pickup""","""No standing""",33,"""Manhattan""","""525""","""175th st""",2013-08-11 18:17:00
"""Passenger""","""New Jersey""","""Delivery Truck…","""No parking""",88,"""Brooklyn""","""100""","""nportland ave""",2013-08-07 04:25:00
"""Passenger""","""New York""","""Sedan""","""No parking""",88,"""Brooklyn""","""100""","""nportland ave""",2013-08-07 04:37:00
"""Passenger""","""New Jersey""","""Sedan""","""No parking""",79,"""Brooklyn""",null,"""760th broadway…",2013-07-18 08:39:00
"""Passenger""","""New York""","""Sedan""","""No parking""",79,"""Brooklyn""",null,"""760th broadway…",2013-07-18 08:45:00


In [ ]:
print('Est. memory_usage, mb: ', int(dfpl.estimated_size()/ 1024 / 1024) )
dfpl.head(5)

Est. memory_usage, mb:  1800


plate_type,registration_state,vehicle_body_type,violation_description,violation_precinct,violation_county,house_number,street_name,issue_datetime
cat,cat,cat,cat,u16,cat,str,str,datetime[μs]
"""Passenger""","""New York""","""Suburban""","""Wrong place""",33,"""Manhattan""","""712""","""175th st""",2013-08-04 07:52:00
"""Commercial""","""New York""","""Van""","""Wrong place""",33,"""Manhattan""","""201""","""177th st""",2013-08-04 12:40:00
"""Commercial""","""New York""","""Pickup""","""Wrong place""",33,"""Manhattan""","""520""","""163rd st""",2013-08-05 12:43:00
"""Commercial""","""New York""","""Van""","""Wrong place""",33,"""Manhattan""","""517""","""176th st""",2013-08-05 14:32:00
"""Passenger""","""New Jersey""","""Pickup""","""No standing""",33,"""Manhattan""","""525""","""175th st""",2013-08-11 18:17:00


## Check precincts for validity

In [ ]:
precinct_geo = 'https://data.cityofnewyork.us/api/geospatial/78dh-3ptz?method=export&format=GeoJSON'

with urlopen(precinct_geo) as response:
    precincts = json.load(response)

In [ ]:
valid_precincts=[]
for i in range(len(precincts['features'])):
  valid_precincts.append(int(precincts['features'][i]['properties']['precinct']))

print(valid_precincts)

[1, 5, 6, 7, 9, 10, 13, 14, 17, 18, 19, 20, 22, 23, 24, 25, 26, 28, 30, 32, 33, 34, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 52, 60, 61, 62, 63, 66, 67, 68, 69, 70, 71, 72, 73, 75, 76, 77, 78, 79, 81, 83, 84, 88, 90, 94, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 120, 121, 122, 123]


In [ ]:
dfpl = dfpl.filter(pl.col('violation_precinct').is_in(valid_precincts))

In [ ]:
print('Est. memory_usage, mb: ', int(dfpl.estimated_size()/ 1024 / 1024) )
dfpl.head(5)

Est. memory_usage, mb:  1800


plate_type,registration_state,vehicle_body_type,violation_description,violation_precinct,violation_county,house_number,street_name,issue_datetime
cat,cat,cat,cat,u16,cat,str,str,datetime[μs]
"""Passenger""","""New York""","""Suburban""","""Wrong place""",33,"""Manhattan""","""712""","""175th st""",2013-08-04 07:52:00
"""Commercial""","""New York""","""Van""","""Wrong place""",33,"""Manhattan""","""201""","""177th st""",2013-08-04 12:40:00
"""Commercial""","""New York""","""Pickup""","""Wrong place""",33,"""Manhattan""","""520""","""163rd st""",2013-08-05 12:43:00
"""Commercial""","""New York""","""Van""","""Wrong place""",33,"""Manhattan""","""517""","""176th st""",2013-08-05 14:32:00
"""Passenger""","""New Jersey""","""Pickup""","""No standing""",33,"""Manhattan""","""525""","""175th st""",2013-08-11 18:17:00


## Top `street_name` in every precinct

In [ ]:
top_streets = sum( # to convert nested list into flat one
    dfpl.group_by(['violation_precinct']).agg(pl.col('street_name').value_counts(sort=True).head(15))['street_name'].to_list(),
[])


top_streets_list = []
for street in top_streets:
    if street['street_name'] not in top_streets_list:
        top_streets_list.append(street['street_name'])

top_streets_list

['broadway',
 'pearl st',
 'church st',
 'greenwich st',
 'warren st',
 'fulton st',
 'front st',
 'nassau st',
 'murray st',
 'b rd st',
 'water st',
 'liberty st',
 'william st',
 'pine st',
 'mercer st',
 '3rd ave',
 'madison ave',
 'lexington ave',
 '1st ave',
 '2nd ave',
 'york ave',
 '86th st',
 '5th ave',
 'park ave',
 '61st st',
 '71st st',
 '70th st',
 '87th st',
 '72nd st',
 '63rd st',
 'bay st',
 'bard ave',
 'victory blvd',
 'st marks pl',
 'forest ave',
 'central ave',
 'castleton ave',
 'othersee comments',
 'hamilton ave',
 'van duzer st',
 'slosson ter',
 'canal st',
 'st pauls ave',
 'ervelt ave',
 'arthur ave',
 'tremont ave',
 '187th st',
 'hughes ave',
 'webster ave',
 'belmont ave',
 'croton ave',
 'belmont lot',
 'prospect ave',
 'hoffman st',
 'southern blvd',
 '179th st',
 '178th st',
 'washington ave',
 'chester ave',
 'crosby ave',
 'buhre ave',
 'baychester ave',
 'chester sq',
 'middletown rd',
 'hutchinson river parky',
 'edison ave',
 'hobart ave',
 'dreis

In [ ]:
len(top_streets_list)

710

In [ ]:
dfpl.filter(~pl.col('street_name').is_in(top_streets_list))

plate_type,registration_state,vehicle_body_type,violation_description,violation_precinct,violation_county,house_number,street_name,issue_datetime
cat,cat,cat,cat,u16,cat,str,str,datetime[μs]
"""Passenger""","""New York""","""Suburban""","""Wrong place""",33,"""Manhattan""","""712""","""175th st""",2013-08-04 07:52:00
"""Commercial""","""New York""","""Van""","""Wrong place""",33,"""Manhattan""","""517""","""176th st""",2013-08-05 14:32:00
"""Passenger""","""New Jersey""","""Pickup""","""No standing""",33,"""Manhattan""","""525""","""175th st""",2013-08-11 18:17:00
"""Passenger""","""New Jersey""","""Delivery Truck…","""No parking""",88,"""Brooklyn""","""100""","""nportland ave""",2013-08-07 04:25:00
"""Passenger""","""New York""","""Sedan""","""No parking""",88,"""Brooklyn""","""100""","""nportland ave""",2013-08-07 04:37:00
"""Passenger""","""New Jersey""","""Sedan""","""No parking""",79,"""Brooklyn""",null,"""760th broadway…",2013-07-18 08:39:00
"""Passenger""","""New York""","""Sedan""","""No parking""",79,"""Brooklyn""",null,"""760th broadway…",2013-07-18 08:45:00
"""Passenger""","""New York""","""Sedan""","""No parking""",79,"""Brooklyn""",null,"""760th broadway…",2013-07-18 09:07:00
"""Passenger""","""New York""","""Sedan""","""No parking""",79,"""Brooklyn""","""760""","""bwy""",2013-08-12 18:56:00


In [ ]:
dfpl = dfpl.with_columns(pl.col('street_name').cast(pl.Utf8).replace(old=top_streets_list, new=top_streets_list, default='other'))

In [ ]:
print('Est. memory_usage, mb: ', int(dfpl.estimated_size()/ 1024 / 1024) )
dfpl.head(5)

Est. memory_usage, mb:  1771


plate_type,registration_state,vehicle_body_type,violation_description,violation_precinct,violation_county,house_number,street_name,issue_datetime
cat,cat,cat,cat,u16,cat,str,str,datetime[μs]
"""Passenger""","""New York""","""Suburban""","""Wrong place""",33,"""Manhattan""","""712""","""other""",2013-08-04 07:52:00
"""Commercial""","""New York""","""Van""","""Wrong place""",33,"""Manhattan""","""201""","""177th st""",2013-08-04 12:40:00
"""Commercial""","""New York""","""Pickup""","""Wrong place""",33,"""Manhattan""","""520""","""163rd st""",2013-08-05 12:43:00
"""Commercial""","""New York""","""Van""","""Wrong place""",33,"""Manhattan""","""517""","""other""",2013-08-05 14:32:00
"""Passenger""","""New Jersey""","""Pickup""","""No standing""",33,"""Manhattan""","""525""","""other""",2013-08-11 18:17:00


In [ ]:
dfpl = dfpl.with_columns(
    issue_date=pl.col("issue_datetime").cast(pl.Date),
    issue_hour=pl.col("issue_datetime").dt.hour()
).drop('issue_datetime')

In [ ]:
dfpl

plate_type,registration_state,vehicle_body_type,violation_description,violation_precinct,violation_county,house_number,street_name,issue_date,issue_hour
cat,cat,cat,cat,u16,cat,str,str,date,i8
"""Passenger""","""New York""","""Suburban""","""Wrong place""",33,"""Manhattan""","""712""","""other""",2013-08-04,7
"""Commercial""","""New York""","""Van""","""Wrong place""",33,"""Manhattan""","""201""","""177th st""",2013-08-04,12
"""Commercial""","""New York""","""Pickup""","""Wrong place""",33,"""Manhattan""","""520""","""163rd st""",2013-08-05,12
"""Commercial""","""New York""","""Van""","""Wrong place""",33,"""Manhattan""","""517""","""other""",2013-08-05,14
"""Passenger""","""New Jersey""","""Pickup""","""No standing""",33,"""Manhattan""","""525""","""other""",2013-08-11,18
"""Passenger""","""New Jersey""","""Delivery Truck…","""No parking""",88,"""Brooklyn""","""100""","""other""",2013-08-07,4
"""Passenger""","""New York""","""Sedan""","""No parking""",88,"""Brooklyn""","""100""","""other""",2013-08-07,4
"""Passenger""","""New Jersey""","""Sedan""","""No parking""",79,"""Brooklyn""",null,"""other""",2013-07-18,8
"""Passenger""","""New York""","""Sedan""","""No parking""",79,"""Brooklyn""",null,"""other""",2013-07-18,8


# Aggregation for Tableau

In [ ]:
# filter between dates

start_date = datetime.datetime(2016, 7, 1)
end_date = datetime.datetime(2017, 6, 30)

dfpl_short = (dfpl.lazy()
    .filter(pl.col("issue_date").is_between(start_date, end_date))
    .collect()
)

In [ ]:
dfpl_tableau = dfpl_short.group_by([
    'plate_type',
    'registration_state',
    'vehicle_body_type',
    'violation_description',
    'violation_precinct',
    'violation_county',
    'street_name',
    'issue_date',
    'issue_hour'
]
              ).agg(pl.col('plate_type').count().alias('violation_count')).sort(by='violation_count', descending=False)

In [ ]:
output_file = 'data_tableau.csv'
dfpl_tableau.write_csv(path+output_file)

In [ ]:
gc.collect()

479